In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
#reading the data into the python notebook in form of a dataframe
my_data = pd.read_csv(r'C:\Users\vishv_ladru5n\Downloads\task2.csv')
my_data["Answer"].value_counts()

No                    1571
Yes                    164
I don't use Tinder     161
Name: Answer, dtype: int64

In [4]:
#check for missing values
print(my_data.isnull().sum())

ID                          0
Segment type                0
Segment Description         0
Answer                      0
Count                       0
Percentage                  0
It became a relationship    0
dtype: int64


In [5]:
#assigning the dependent variable to y and dropping the answer column as it is inconsistent with the dependent variable
y=my_data['It became a relationship']
object_cols=['Segment type']
X=my_data.drop(['It became a relationship','Answer'],axis=1)

In [6]:
#splitting the dataframe into train and test sets
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0,train_size =0.8,test_size=0.2)

In [7]:
#one hot encoding categorical column
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_X[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(val_X[object_cols]))
OH_cols_train.index = train_X.index
OH_cols_valid.index = val_X.index
num_X_train=train_X.drop(['Segment Description','Segment type'],axis=1)
num_X_valid=val_X.drop(['Segment Description','Segment type'],axis=1)
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

In [8]:
#RandomForestClassifiermodel
model_2= RandomForestClassifier(n_estimators=50)
model_2.fit(OH_X_train,train_y)
from sklearn.metrics import mean_absolute_error
predicted=model_2.predict(OH_X_valid)
print("accuracy :",(accuracy_score(val_y, predicted)*100),"%")

accuracy : 91.84210526315789 %


In [9]:
#XGBClassifier
from xgboost import XGBClassifier
my_model = XGBClassifier(n_estimators=50, learning_rate=0.5,n_jobs=5,use_label_encoder=False)
my_model.fit(OH_X_train, train_y, 
             early_stopping_rounds=4, 
             eval_set=[(OH_X_valid,val_y)],eval_metric='logloss', 
             verbose=False)
predicted=my_model.predict(OH_X_valid)
print("accuracy :",(accuracy_score(val_y, predicted)*100),"%")

accuracy : 91.57894736842105 %
